In [5]:
from pyomo.environ import * 
from pyomo.opt import SolverFactory
import numpy as np
import matplotlib.pyplot as plt

In [8]:
def lagrangian_branch_and_cut(scenario_models, initial_multipliers, UB=10, max_iter=10, tolerance=1e-3):
    """
    Parameters:
        scenario_models (list): model for each scenario
        initial_multipliers (dict): [lambdas_x:xxx,lambdas_y:yyy]
        UB (float): current upper bound
        max_iter (float): maximum iteration times
        tolerance (float): decide when to stop

    Returns: LB (float): the calculated lower bound
    """
    N = len(scenario_models)
    lambdas_x = initial_multipliers['lambda_x']
    lambdas_y = initial_multipliers['lambda_y']

    def scenario_objective(n, submodel, x_n, y_n, lambdas_x, lambdas_y):
        # Lagrangean penalty terms
        lx = lambdas_x[n - 1] if n <= N - 1 else 0
        lx_prev = lambdas_x[n - 2] if n > 1 else 0
        ly = lambdas_y[n - 1] if n <= N - 1 else 0
        ly_prev = lambdas_y[n - 2] if n > 1 else 0
        weight = 1 / N
        return weight * submodel.obj_base + (lx - lx_prev) * x_n + (ly - ly_prev) * y_n

    for _ in range(max_iter):
        model = ConcreteModel()
        model.N = RangeSet(1, N)
        model.x = Var(model.N, within=Reals, bounds=(0, 10)) #fix the bound here!
        model.y = Var(model.N, within=Binary)                #fix the bound here!
        model.submodels = Block(model.N)

        for n in model.N:
            model.submodels[n].transfer_attributes_from(scenario_models[n-1])

        def coupling_x_rule(m, n):
            if n < N:
                return m.x[n] == m.x[n+1]
            return Constraint.Skip
        model.couple_x = Constraint(model.N, rule=coupling_x_rule)

        def coupling_y_rule(m, n):
            if n < N:
                return m.y[n] == m.y[n+1]
            return Constraint.Skip
        model.couple_y = Constraint(model.N, rule=coupling_y_rule)

        model.obj = Objective(expr=sum(
            scenario_objective(n, model.submodels[n], model.x[n], model.y[n], lambdas_x, lambdas_y)
            for n in model.N
        ), sense=minimize)

        solver = SolverFactory('scip', executable='/Users/yinhuang/miniconda3/bin/scip')
        solver.solve(model)

        # get x_n, y_n values from solution
        x_vals = [value(model.x[n]) for n in model.N]
        y_vals = [value(model.y[n]) for n in model.N]
        LB = value(model.obj)

        # update lambda
        denom = sum((x_vals[n] - x_vals[n+1])**2 + (y_vals[n] - y_vals[n+1])**2 for n in range(N - 1))
        step = 0.5 * (UB - LB) / denom if denom > 1e-8 else 0.0
        lambdas_x = [lambdas_x[n] + step * (x_vals[n] - x_vals[n+1]) for n in range(N - 1)]
        lambdas_y = [lambdas_y[n] + step * (y_vals[n] - y_vals[n+1]) for n in range(N - 1)]

        # convergence check
        gap = (UB - LB) / UB if UB != 0 else float('inf')
        if gap <= tolerance:
            break

    return LB


In [9]:
def build_scenario_model():
    m = ConcreteModel()
    m.x = Var(bounds=(0, 10))
    m.y = Var(within=Binary)
    m.u = Var(bounds=(0, 5))
    m.v = Var(within=Binary)
    m.con1 = Constraint(expr=m.u + m.v <= 4)
    m.con2 = Constraint(expr=m.x >= m.u)
    m.con3 = Constraint(expr=m.y >= m.v)
    m.obj_base = Expression(expr=0.5 * m.x + m.y + m.u + 2 * m.v)
    return m

N = 3
scenario_models = [build_scenario_model() for _ in range(N)]
initial_multipliers = {'lambda_x': [0.0 for _ in range(N-1)], 'lambda_y': [0.0 for _ in range(N-1)]}
opt_value = lagrangian_branch_and_cut(scenario_models, initial_multipliers)
print("Optimized lower bound:", opt_value)


AttributeError: 'BlockData' object has no attribute 'obj_base'